In [1]:
import sys, os
from dotenv import load_dotenv
import argparse
import json
from typing import Dict, List
import logging
import GraphDB.utils as utils
from GraphDB.LegalGraphDB import LegalGraphDB

load_dotenv(verbose=True)

# 로깅 설정 (INFO 레벨)
logging.basicConfig(level=logging.INFO)
path = os.getcwd()
root_path = os.path.dirname(path)

# User Query answer by gpt-4o (No documents)
- Graph Data Science -> Neo4j plugin 설치 필요

In [ ]:
# ## Query list
# SYSTEM_PROMPT = "당신은 자본시장법 법률 전문가입니다. 가지고 있는 지식을 최대한 활용하여 사용자의 질문에 답변하세요."

# QUERY_1 = "증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?"
# QUERY_2 = "회사채의 무보증 후순위사채 기업실사를 규정하고 있는데, 기업실사에서 회사의 기관 및 계열회사에 관한 사항은 어떤 것을 확인해야 하나?"
# QUERY_3 = "증권신고서의 효력 발생기간은?"
# QUERY_4 = "증권신고서 발행을 위해서는 어떤 첨부서류가 필요한가?"
# QUERY_5 = "정정신고가 증권신고서에 어떤 효력을 미치는가?"
# QUERY_6 = "증권신고서와 투자설명서의 부실기재에 대한 과징금이 부과되는 경우 부과대상과 부과금액은?"
# QUERY_7 = "정정신고서를 부실기재했을 경우 과징금은?"
# QUERY_8 = "증권신고서 부실기재 과징금은?"

# ## Query embedding
# QUERY_1_EMB = utils.text_embed(QUERY_1)
# QUERY_2_EMB = utils.text_embed(QUERY_2)
# QUERY_3_EMB = utils.text_embed(QUERY_3)
# QUERY_4_EMB = utils.text_embed(QUERY_4)
# QUERY_5_EMB = utils.text_embed(QUERY_5)
# QUERY_6_EMB = utils.text_embed(QUERY_6)
# QUERY_7_EMB = utils.text_embed(QUERY_7)
# QUERY_8_EMB = utils.text_embed(QUERY_8)

# ## Keyword Extraction
# KEYWORDS_1 = utils.extract_user_keyword(QUERY_1)
# KEYWORDS_2 = utils.extract_user_keyword(QUERY_2)
# KEYWORDS_3 = utils.extract_user_keyword(QUERY_3)
# KEYWORDS_4 = utils.extract_user_keyword(QUERY_4)
# KEYWORDS_5 = utils.extract_user_keyword(QUERY_5)
# KEYWORDS_6 = utils.extract_user_keyword(QUERY_6)
# KEYWORDS_7 = utils.extract_user_keyword(QUERY_7)
# KEYWORDS_8 = utils.extract_user_keyword(QUERY_8)

# ## GPT Answer
# ANSWER_1 = utils.get_gpt_answer(QUERY_1, SYSTEM_PROMPT)
# ANSWER_2 = utils.get_gpt_answer(QUERY_2, SYSTEM_PROMPT)
# ANSWER_3 = utils.get_gpt_answer(QUERY_3, SYSTEM_PROMPT)
# ANSWER_4 = utils.get_gpt_answer(QUERY_4, SYSTEM_PROMPT)
# ANSWER_5 = utils.get_gpt_answer(QUERY_5, SYSTEM_PROMPT)
# ANSWER_6 = utils.get_gpt_answer(QUERY_6, SYSTEM_PROMPT)
# ANSWER_7 = utils.get_gpt_answer(QUERY_7, SYSTEM_PROMPT)
# ANSWER_8 = utils.get_gpt_answer(QUERY_8, SYSTEM_PROMPT)


# ## Answer embedding
# ANSWER_1_EMB = utils.text_embed(ANSWER_1)
# ANSWER_2_EMB = utils.text_embed(ANSWER_2)
# ANSWER_3_EMB = utils.text_embed(ANSWER_3)
# ANSWER_4_EMB = utils.text_embed(ANSWER_4)
# ANSWER_5_EMB = utils.text_embed(ANSWER_5)
# ANSWER_6_EMB = utils.text_embed(ANSWER_6)
# ANSWER_7_EMB = utils.text_embed(ANSWER_7)
# ANSWER_8_EMB = utils.text_embed(ANSWER_8)

# clause 탐색 함수 정의

In [2]:
config_path = os.path.join(root_path, 'codes', 'configs', 'config.json')

with open(config_path, 'r') as f:
    config = json.load(f)

dbms = LegalGraphDB(auradb=False, config=config)

## Neo4j AuraDB 서버와의 연결이 성공적으로 확인되었습니다.


In [3]:
import time
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os

# Neo4j에서 쿼리 벡터와 유사한 노드 검색
def find_similar_nodes(query_embedding_vec, config, top_k):
    retrieved_nodes = []  # 중복 방지를 위한 리스트 사용
    top_k = config.get("top_k", top_k)  # config에서 top_k 값을 가져오며 기본값을 5로 설정
    query = f"""
    WITH $embedding AS query_embedding_vec
    MATCH (n)
    WHERE ANY(label IN labels(n) WHERE label STARTS WITH 'Clause')
    WITH n, gds.similarity.cosine(n.embedding, query_embedding_vec) AS similarity
    ORDER BY similarity DESC
    LIMIT {top_k}
    RETURN n, similarity
    """

    with dbms.driver.session(database=config["database"]) as session:
        result = session.run(query, embedding=query_embedding_vec)  

        for record in result:
            node = record['n']
            node_id = node.element_id  # 노드의 고유 식별자를 사용하여 중복 방지
            similarity = record['similarity']
            labels = list(node.labels)[0] if len(node.labels) > 0 else "Unknown"  # 리스트 형태에서 첫 번째 레이블만 추출
            retrieved_nodes.append({
                'node_id': node_id,
                'index': node.get('law_index'),
                'labels': labels,
                'text': node.get('text'),
                'similarity': similarity,
                'hop': 0  # Top K 노드는 hop 0으로 표시
            })
    
    return retrieved_nodes


In [4]:
# 현재 키워드 삭제된 상태
# Neo4j에서 유사한 노드를 검색하고 refers_to 엣지를 따라 연결된 노드를 탐색하는 함수

def find_related_nodes_with_keywords(query_embedding_vec, keywords, config, top_k, hop):
    # Top K 유사한 노드 검색
    top_k_nodes = find_similar_nodes(query_embedding_vec, config, top_k)

    # Top K 노드의 element_id를 추출하여 이웃 노드 탐색 시작
    top_k_element_ids = [node['node_id'] for node in top_k_nodes]
    visited_nodes = set(top_k_element_ids)  # 중복된 노드 방문 방지

    # Top K 노드를 관련 문서 집합에 추가
    related_documents = set()
    for node in top_k_nodes:
        related_documents.add(frozenset(node.items()))  

    def recursive_search(element_ids, current_hop):
        if current_hop > hop:
            return set()  

        related_nodes = set()  
        for element_id in element_ids:
            query = """
            MATCH (n)-[:refers_to]->(neighbor)
            WHERE elementId(n) = $element_id
            RETURN neighbor
            """
            with dbms.driver.session(database=config["database"]) as session:
                result = session.run(query, element_id=element_id)
                for record in result:
                    neighbor = record['neighbor']
                    neighbor_id = neighbor.element_id  
                    
                    # 이미 방문한 노드 제외 
                    if neighbor_id in visited_nodes:
                        continue

                    visited_nodes.add(neighbor_id)
                    labels = list(neighbor.labels)[0] if len(neighbor.labels) > 0 else "Unknown"

                    # 이웃 노드의 embedding을 가져와 유사도 계산
                    neighbor_embedding = np.array(neighbor['embedding'])
                    similarity = cosine_similarity([query_embedding_vec], [neighbor_embedding])[0][0]
                    
                    # 유사도와 키워드 조건을 만족하는지 확인
                    if similarity >= config['threshold']:
                        related_node = {
                            'node_id': neighbor_id,
                            'index': neighbor.get('law_index'),
                            'labels': labels,
                            'text': neighbor.get('text'),
                            'similarity': similarity,
                            'hop': current_hop
                        }
                        related_nodes.add(frozenset(related_node.items()))  # frozenset으로 변환 후 추가하여 중복 방지
                        
                        # 재귀적으로 연결된 노드를 탐색 (hop을 올바르게 증가시켜 호출)
                        related_nodes.update(recursive_search([neighbor_id], current_hop + 1))
        return related_nodes

    # 이웃 노드를 탐색하고 유사도와 키워드 필터링 적용
    for element_id in top_k_element_ids:
        related_documents.update(recursive_search([element_id], current_hop=1))

    related_documents = [dict(doc) for doc in related_documents]  # frozenset을 dict로 변환하여 리스트로 정렬
    related_documents = sorted(related_documents, key=lambda x: x['similarity'], reverse=True)

    return top_k_nodes, related_documents

## Answer와 유사한 document 찾기

- 키워드는 그대로 User Query에서 추출
- Answer와 유사한 top-k document 찾아보기
- Keyword retrieve + clause retrieve 합치고 탐색 시간 , 탐색 결과 json 파일 형식으로 저장

In [5]:
## Query list
SYSTEM_PROMPT = "당신은 자본시장법 법률 전문가입니다. 가지고 있는 지식을 최대한 활용하여 사용자의 질문에 답변하세요."

# QUERY_1 = "증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?"
# QUERY_2 = "회사채의 무보증 후순위사채 기업실사를 규정하고 있는데, 기업실사에서 회사의 기관 및 계열회사에 관한 사항은 어떤 것을 확인해야 하나?"
# QUERY_3 = "증권신고서의 효력 발생기간은?"
# QUERY_4 = "증권신고서 발행을 위해서는 어떤 첨부서류가 필요한가?"
# QUERY_5 = "정정신고가 증권신고서에 어떤 효력을 미치는가?"
# QUERY_6 = "증권신고서와 투자설명서의 부실기재에 대한 과징금이 부과되는 경우 부과대상과 부과금액은?"
# QUERY_7 = "정정신고서를 부실기재했을 경우 과징금은?"
# QUERY_8 = "증권신고서 부실기재 과징금은?"

In [6]:
from datetime import datetime

def retrieve(query, top_k, hop, search_type):
    # query & query embed
    query_embed = utils.text_embed(query)
    query_keyword = utils.extract_user_keyword(query)

    print('## input query : ', query)
    print('## query keyword : ', query_keyword)

    # Retrieve keyword triplets
    k_start_time = time.time()
    relevant_keywords = dbms.get_all_triplet(query, query_keyword)

    k_end_time = time.time()
    k_total_time = round(k_end_time - k_start_time, 4)

    print(f"## Total time taken: {k_total_time} seconds\n")
    print(f'## keyword triplets | total : {len(relevant_keywords)} ##')

    for triplet in relevant_keywords:
        print(triplet)

    # gpt answer
    gpt_answer = utils.get_gpt_answer(query, SYSTEM_PROMPT)
    print('## answer : ', gpt_answer)

    gpt_answer_embed = utils.text_embed(gpt_answer)


    # Retrieve clause
    c_start_time = time.time()
    if search_type == "query":
        top_k_nodes, relevant_nodes = find_related_nodes_with_keywords(query_embed, query_keyword, config, top_k, hop)
    elif search_type == "answer":
        top_k_nodes, relevant_nodes = find_related_nodes_with_keywords(gpt_answer_embed, query_keyword, config, top_k, hop)
    else:
        raise ValueError("## Wrong Search type ##")

    c_end_time = time.time()
    c_total_time = round(c_end_time - c_start_time, 4)

    print(f"## Total time taken: {c_total_time} seconds\n")
    print(f'## top-k : {top_k} | relevent nodes : {len(relevant_nodes)}')
    

    max_length = min(len(top_k_nodes), len(relevant_nodes))
    print(f'## relevant documents | total : {max_length}##')

    for idx in range(max_length):
        top_k_node = top_k_nodes[idx]
        related_doc = relevant_nodes[idx]
        
        print(f"\n####### idx: {idx} #######")
        print(f"## VECTOR ## : Labels: {top_k_node['labels']}, Index: {top_k_node['index']}, Similarity: {top_k_node['similarity']:.4f}")
        print(f"## HOP ## : hops : {related_doc['hop']}, Labels: {related_doc['labels']}, Index: {related_doc['index']}, Similarity: {related_doc['similarity']:.4f}")


    json_dict = {
        "query" : query,
        "query_keyword" : query_keyword,
        "gpt_answer": gpt_answer,
        "keyword_total_time" : k_total_time,
        "clause_total_time" : c_total_time,
        "total_time" : k_total_time + c_total_time,
        "relevant_keywords" : relevant_keywords,
        "relevant_nodes" : relevant_nodes,
        "query_embed" : query_embed,
        "gpt_answer_embed" : gpt_answer_embed,
    }

    # result 저장
    # Get the current date and time
    current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"{search_type}_topk_{top_k}_hop_{hop}_{current_datetime}.json"

    path = os.getcwd()
    root_path = os.path.dirname(path)
    result_path = os.path.join(root_path, 'results', '1-2', 'retrieve', file_name)

    with open(result_path, "w") as f:
        json.dump(json_dict, f, indent=4, ensure_ascii=False)
    
    print('## result json file saved in', result_path)
    
    return json_dict

hop 결과 탐색을 위한 json format

> {query}\_topk\_{topk num}\_hop\_{hop num}.json

```json
"query" : 사용자 쿼리,
"query_embed" : 사용자 쿼리의 임베딩,
"query_keyword" : 사용자 쿼리의 키워드,
"gpt_answer" : GPT의 답변 (근거 문서 x),
"gpt_answer_embed" : GPT 답변의 임베딩,
"keyword_total_time" : keyword triplets retrieve에 걸린 시간,
"clause_total_time" : relevant clause retrieve에 걸린 시간
"total_time" : keyword_total_time + clause_total_time,
"relevant_keywords" : [
    "(정정신고서 -[INCLUDED_IN]-> 증권신고서) IN 법_제129조_신고서와 보고서의 공시",
    "(정정신고서 -[SAME_AS]-> 공개매수신고서의 기재내용을 정정한 신고서) IN 법_제136조제1항_정정신고ㆍ공고 등"
]
"relevant_nodes" : [
        {"node_id": "4:82f3945f-31c1-4c80-98df-efa9a14a3bbd:5165",
        "index": "제125조제1항",
        "labels": "Clause_01_law_main",
        "text": "증권신고서(정정신고서 및 첨부서류를 포함한다. 이하 이 조에서 같다)...",
        "similarity": 0.7361304001909146,
        "hop": 0}
        
    ]

```

In [7]:
QUERY_1 = "증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?"

In [8]:
QUERY_1_dict = retrieve(QUERY_1, config['top_k'], config['hop'], "query")

## config file loaded ##
{'model': 'gpt-4o', 'embedding_model': 'text-embedding-3-large', 'chunk_size': 1024, 'chunk_overlap': 50, 'top_k': 5, 'prev_turns': 5, 'hop': 3, 'rag': 'Graph', 'keyword_prompt_path': '../codes/GraphDB/prompt/triplet_extractor_cot.txt', 'database': 'legal-graph', 'query_keyword_prompt_path': '../codes/GraphDB/prompt/query_keyword_extractor.txt', 'threshold': 0.4, 'max_tokens': 1024}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


## Query : 증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?/n/n## Answer : 증권신고서 | 정정신고서 | 기재 | 투자자 | 배상책임 | 조문
## input query :  증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?
## query keyword :  ['증권신고서', '정정신고서', '기재', '투자자', '배상책임', '조문']
## User query :  증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?
Keyword :  ['증권신고서', '정정신고서', '기재', '투자자', '배상책임', '조문']
## Total time taken: 0.0392 seconds

## keyword triplets | total : 20 ##
(투자자 -[INCLUDED_IN]-> 투자매매업자) IN 시행령_제208조제2항_공매도의 제한
(투자자 -[INCLUDED_IN]-> 임직원) IN 시행령_제99조제4항_불건전 영업행위의 금지
(증권신고서 -[INCLUDED_IN]-> 정정신고서) IN 시행령_제68조제5항_불건전 영업행위의 금지
(투자자 -[SAME_AS]-> 적격투자자) IN 법_제249조의2_일반 사모집합투자기구의 투자자
(투자자 -[INCLUDED_IN]-> 일반 사모집합투자기구) IN 시행령_제14조제2항_사모집합투자기구의 기준
(투자자 -[INCLUDED_IN]-> 일반 사모집합투자기구의 투자자) IN 법_제249조의8제4항_일반 사모집합투자기구에 대한 특례
(투자자 -[INCLUDED_IN]-> 투자신탁재산을 운용하는 일반 사모집합투자업자) IN 법_제249조의8제4항_일반 사모

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


## Query : 증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?/n/n## Answer : 증권신고서 또는 정정신고서에 거짓의 기재 또는 표시가 있거나 중요한 사항이 기재 또는 표시되지 않음으로 인해 투자자가 손해를 입은 경우, 그 배상책임의 근거가 되는 조문은 '자본시장과 금융투자업에 관한 법률(자본시장법)' 제125조입니다. 

이 조항은 발행인, 기타 보고의무자, 또는 그들의 이사나 임원이 증권신고서 또는 정정신고서의 허위기재나 중요한 사항의 누락으로 인해 발생한 손해에 대해 배상책임을 지는 것으로 명시하고 있습니다. 투자자는 이러한 허위기재나 누락으로 인한 손해를 입었을 때 해당 조항을 근거로 손해배상 청구를 할 수 있습니다.
## answer :  증권신고서 또는 정정신고서에 거짓의 기재 또는 표시가 있거나 중요한 사항이 기재 또는 표시되지 않음으로 인해 투자자가 손해를 입은 경우, 그 배상책임의 근거가 되는 조문은 '자본시장과 금융투자업에 관한 법률(자본시장법)' 제125조입니다. 

이 조항은 발행인, 기타 보고의무자, 또는 그들의 이사나 임원이 증권신고서 또는 정정신고서의 허위기재나 중요한 사항의 누락으로 인해 발생한 손해에 대해 배상책임을 지는 것으로 명시하고 있습니다. 투자자는 이러한 허위기재나 누락으로 인한 손해를 입었을 때 해당 조항을 근거로 손해배상 청구를 할 수 있습니다.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## Total time taken: 4.9714 seconds

## top-k : 5 | relevent nodes : 9
## relevant documents | total : 5##

####### idx: 0 #######
## VECTOR ## : Labels: Clause_01_law_main, Index: 제125조제1항, Similarity: 0.7338
## HOP ## : hops : 0, Labels: Clause_01_law_main, Index: 제125조제1항, Similarity: 0.7338

####### idx: 1 #######
## VECTOR ## : Labels: Clause_01_law_main, Index: 제117조의12제1항, Similarity: 0.6849
## HOP ## : hops : 0, Labels: Clause_01_law_main, Index: 제117조의12제1항, Similarity: 0.6849

####### idx: 2 #######
## VECTOR ## : Labels: Clause_01_law_main, Index: 제122조제1항, Similarity: 0.6635
## HOP ## : hops : 0, Labels: Clause_01_law_main, Index: 제122조제1항, Similarity: 0.6635

####### idx: 3 #######
## VECTOR ## : Labels: Clause_01_law_main, Index: 제162조제1항, Similarity: 0.6533
## HOP ## : hops : 0, Labels: Clause_01_law_main, Index: 제162조제1항, Similarity: 0.6533

####### idx: 4 #######
## VECTOR ## : Labels: Clause_01_law_main, Index: 제142조제1항, Similarity: 0.6529
## HOP ## : hops : 0, Labels

In [9]:
ANSWER_1_dict = retrieve(QUERY_1, config['top_k'], config['hop'], "answer")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


## Query : 증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?/n/n## Answer : 증권신고서 | 정정신고서 | 거짓 기재 | 표시 | 배상책임 | 조문
## input query :  증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?
## query keyword :  ['증권신고서', '정정신고서', '거짓 기재', '표시', '배상책임', '조문']
## User query :  증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?
Keyword :  ['증권신고서', '정정신고서', '거짓 기재', '표시', '배상책임', '조문']
## Total time taken: 0.0305 seconds

## keyword triplets | total : 11 ##
(증권신고서 -[INCLUDED_IN]-> 첨부서류) IN 시행령_제68조제5항_불건전 영업행위의 금지
(정정신고서 -[INCLUDED_IN]-> 일괄신고서) IN 시행령_제122조제3항_일괄신고추가서류 등
(증권신고서 -[INCLUDED_IN]-> 일괄신고추가서류) IN 법_제123조제2항_투자설명서의 작성ㆍ공시
(증권신고서 -[SAME_AS]-> 법 제119조제3항에 따른 증권신고서) IN 시행령_제124조_증권신고서에 대한 대표이사 등의 확인ㆍ검토
(정정신고서 -[INCLUDED_IN]-> 증권신고서) IN 시행령_제68조제5항_불건전 영업행위의 금지
(정정신고서 -[INCLUDED_IN]-> 첨부서류) IN 법_제142조제1항_공개매수자 등의 배상책임
(증권신고서 -[SAME_AS]-> 일괄신고서) IN 법_제119조제3항_모집 또는 매출의 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


## Query : 증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?/n/n## Answer : 자본시장과 금융투자업에 관한 법률에 따르면, 증권신고서 또는 정정신고서에 거짓 기재, 표시가 있거나 중요한 사항이 누락되어 투자자가 손해를 입은 경우 발행인, 대표이사, 발행인을 위하여 증권의 매출을 주선한 자, 증권신고서 제출시 또는 효력발생후 1년 이내에 증권을 인수한 자, 감사인 및 증권신고서에 기재된 변호사나 회계사와 같은 전문가들이 투자자에게 손해배상 책임을 질 수 있습니다. 이와 관련된 조문은 주로 자본시장법 제125조에서 제128조에 규정되어 있습니다. 특히, 제125조는 과실책임을, 제126조는 무과실책임을 규정하고 있습니다.
## answer :  자본시장과 금융투자업에 관한 법률에 따르면, 증권신고서 또는 정정신고서에 거짓 기재, 표시가 있거나 중요한 사항이 누락되어 투자자가 손해를 입은 경우 발행인, 대표이사, 발행인을 위하여 증권의 매출을 주선한 자, 증권신고서 제출시 또는 효력발생후 1년 이내에 증권을 인수한 자, 감사인 및 증권신고서에 기재된 변호사나 회계사와 같은 전문가들이 투자자에게 손해배상 책임을 질 수 있습니다. 이와 관련된 조문은 주로 자본시장법 제125조에서 제128조에 규정되어 있습니다. 특히, 제125조는 과실책임을, 제126조는 무과실책임을 규정하고 있습니다.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## Total time taken: 1.0596 seconds

## top-k : 5 | relevent nodes : 26
## relevant documents | total : 5##

####### idx: 0 #######
## VECTOR ## : Labels: Clause_01_law_main, Index: 제125조제1항, Similarity: 0.7837
## HOP ## : hops : 0, Labels: Clause_01_law_main, Index: 제125조제1항, Similarity: 0.7837

####### idx: 1 #######
## VECTOR ## : Labels: Clause_01_law_main, Index: 제117조의12제1항, Similarity: 0.7608
## HOP ## : hops : 0, Labels: Clause_01_law_main, Index: 제117조의12제1항, Similarity: 0.7608

####### idx: 2 #######
## VECTOR ## : Labels: Clause_01_law_main, Index: 제162조제1항, Similarity: 0.7345
## HOP ## : hops : 0, Labels: Clause_01_law_main, Index: 제162조제1항, Similarity: 0.7345

####### idx: 3 #######
## VECTOR ## : Labels: Clause_01_law_main, Index: 제142조제1항, Similarity: 0.7186
## HOP ## : hops : 0, Labels: Clause_01_law_main, Index: 제142조제1항, Similarity: 0.7186

####### idx: 4 #######
## VECTOR ## : Labels: Clause_01_law_main, Index: 제64조제1항, Similarity: 0.6765
## HOP ## : hops : 0, Labels

## json 확인 (query vs answer)

> Query : 증권신고서 또는 정정신고서 중 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니함으로써 투자자가 손해를 입은 경우 배상책임의 근거가 되는 조문은?

> Answer : 자본시장과 금융투자업에 관한 법률(자본시장법)에서 증권신고서 또는 정정신고서의 거짓 기재나 중요사항 누락으로 인한 손해배상 책임에 대한 근거는 제125조에 명시되어 있습니다. \n\n이 조문에 따르면, 허위 기재나 중요한 사항의 누락으로 인해 투자자가 손해를 입은 경우, 증권신고서 작성에 관여한 발행인, 대표이사, 공인회계사, 주간사 등 관련자들이 연대하여 그 손해를 배상할 책임을 지게 됩니다. 이는 투자자 보호를 강화하기 위한 장치로, 책임 주체와 책임 범위를 명확히 하고 있습니다.

- Retrieved time (clause)
    - query: 0.9087
    - answer: 5.2959

- Document (hop=0)
    - Query
        1. 제125조제1항 0.73
        2. 제117조의12제1항 0.68
        3. **제122조제1항** 0.66
        4. 제162조제1항 0.65
        5. 제142조제1항 0.65
    
    - Answer
        1. 제125조제1항 0.74
        2. 제117조의12제1항 0.72
        3. 제162조제1항 0.68
        4. **제64조제1항** 0.67
        5. 제142조제1항 0.62

- 제122조제1항: 금융위원회는 증권신고서의 형식을 제대로 갖추지 아니한 경우 또는 그 증권신고서 중 중요사항에 관하여 거짓의 기재 또는 표시가 있거나 중요사항이 기재 또는 표시되지 아니한 경우와 중요사항의 기재나 표시내용이 불분명하여 투자자의 합리적인 투자판단을 저해하거나 투자자에게 중대한 오해를 일으킬 수 있는 경우에는 그 증권신고서에 기재된 증권의 취득 또는 매수의 청약일 전일까지 그 이유를 제시하고 그 증권신고서의 기재내용을 정정한 신고서(이하 이 장에서 \"정정신고서\"라 한다)의 제출을 요구할 수 있다.

- 제64조제1항: 금융투자업자는 법령ㆍ약관ㆍ집합투자규약ㆍ투자설명서(제123조제1항에 따른 투자설명서를 말한다)에 위반하는 행위를 하거나 그 업무를 소홀히 하여 투자자에게 손해를 발생시킨 경우에는 그 손해를 배상할 책임이 있다. 다만, 배상의 책임을 질 금융투자업자가 제37조제2항, 제44조, 제45조, 제71조 또는 제85조를 위반한 경우(투자매매업 또는 투자중개업과 집합투자업을 함께 영위함에 따라 발생하는 이해상충과 관련된 경우에 한한다)로서 그 금융투자업자가 상당한 주의를 하였음을 증명하거나 투자자가 금융투자상품의 매매, 그 밖의 거래를 할 때에 그 사실을 안 경우에는 배상의 책임을 지지 아니한다.

In [10]:
QUERY_2 = "회사채의 무보증 후순위사채 기업실사를 규정하고 있는데, 기업실사에서 회사의 기관 및 계열회사에 관한 사항은 어떤 것을 확인해야 하나?"
QUERY_2_dict = retrieve(QUERY_2, config['top_k'], config['hop'], "query")
ANSWER_2_dict = retrieve(QUERY_2, config['top_k'], config['hop'], "answer")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


## Query : 회사채의 무보증 후순위사채 기업실사를 규정하고 있는데, 기업실사에서 회사의 기관 및 계열회사에 관한 사항은 어떤 것을 확인해야 하나?/n/n## Answer : 무보증 후순위사채 | 기업실사 | 기관 | 계열회사 | 확인사항
## input query :  회사채의 무보증 후순위사채 기업실사를 규정하고 있는데, 기업실사에서 회사의 기관 및 계열회사에 관한 사항은 어떤 것을 확인해야 하나?
## query keyword :  ['무보증 후순위사채', '기업실사', '기관', '계열회사', '확인사항']
## User query :  회사채의 무보증 후순위사채 기업실사를 규정하고 있는데, 기업실사에서 회사의 기관 및 계열회사에 관한 사항은 어떤 것을 확인해야 하나?
Keyword :  ['무보증 후순위사채', '기업실사', '기관', '계열회사', '확인사항']
## Total time taken: 0.0295 seconds

## keyword triplets | total : 10 ##
(기관 -[SAME_AS]-> 중앙기록관리기관) IN 법_제117조의13제1항_중앙기록관리기관
(계열회사 -[INCLUDED_IN]-> 관계 투자매매업자) IN 시행령_제89조제2항_의결권행사의 제한 등
(계열회사 -[INCLUDED_IN]-> 임직원) IN 시행령_제84조_이해관계인의 범위
(계열회사 -[INCLUDED_IN]-> 대량취득ㆍ처분을 하려는 자) IN 법_제174조제3항_미공개중요정보 이용행위 금지
(계열회사 -[INCLUDED_IN]-> 공개매수예정자) IN 법_제174조제2항_미공개중요정보 이용행위 금지
(계열회사 -[INCLUDED_IN]-> 집합투자업자) IN 시행령_제209조_집합투자기구의 등록요건
(기관 -[INCLUDED_IN]-> 중앙기록관리기관대통령령으로 정하는 바에 따라 온라인소액투자중개업자로부터 온라인소액증권발행인과 투자자에 대한 정보를 제공받아 관리하는) IN 법_제117조의13제1항_중앙기록관리기관
(계열회사 -[INCL

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


## Query : 회사채의 무보증 후순위사채 기업실사를 규정하고 있는데, 기업실사에서 회사의 기관 및 계열회사에 관한 사항은 어떤 것을 확인해야 하나?/n/n## Answer : 회사채의 무보증 후순위사채 발행과 관련한 기업실사를 진행할 때, 회사의 기관 및 계열회사와 관련해서는 다음과 같은 사항들을 확인하는 것이 일반적입니다. 이러한 사항들은 회사의 재무 상태, 운영 구조, 리스크 평가 등에 중요하게 작용할 수 있습니다.

1. **조직 구조**: 회사의 조직도 및 관리 체계, 의사결정 절차 등을 확인합니다. 이는 회사의 경영 효율성과 내부 통제 여부를 평가하는 데 중요합니다.

2. **이사회 및 경영진**: 이사회의 구성, 주요 경영진의 경력 및 평판, 내부 통제 및 지배구조의 투명성 등을 점검합니다.

3. **계열회사 구조**: 계열회사 및 자회사 목록과 지배 구조를 확인하고, 각 계열회사의 재무 상태 및 전략적 중요성을 평가합니다.

4. **관련 당사자 거래**: 회사가 계열회사와 체결한 주요 계약 및 거래 내역을 검토합니다. 특히, 거래가 정상적인 상업 조건에서 이루어졌는지, 투명성이 있는지를 중점적으로 봅니다.

5. **규제 및 법적 환경**: 계열회사가 속한 산업에 관련된 규제 사항, 해당 법적 위험 요인 등을 파악합니다.

6. **재무 상황**: 계열회사의 재무제표 분석을 통해 재무 건전성, 부채 구조, 수익성을 파악하고, 모회사에 미치는 재정적 영향을 평가합니다.

7. **리스크 평가**: 각 계열회사가 당면하고 있는 주요 리스크 요인과 이러한 리스크가 모회사 또는 전체 그룹에 미치는 영향을 분석합니다.

이 외에도, 회사별 특성에 따라 추가적인 조사가 필요할 수 있으며, 특정 규제기관의 요구나 산업 특성에 따라 확인해야 할 추가 항목이 있을 수 있습니다. 기업실사는 철저하게 이루어져야 하며, 필요한 경우 외부 전문가를 통해 심도 있는 분석이 필요할 수 있습니다.
## answer :  회사채의 무보증 후순위사채 발행과 관련한 기업

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


...

## Query : 회사채의 무보증 후순위사채 기업실사를 규정하고 있는데, 기업실사에서 회사의 기관 및 계열회사에 관한 사항은 어떤 것을 확인해야 하나?/n/n## Answer : 무보증 후순위사채 | 기업실사 | 기관 | 계열회사 | 확인사항
## input query :  회사채의 무보증 후순위사채 기업실사를 규정하고 있는데, 기업실사에서 회사의 기관 및 계열회사에 관한 사항은 어떤 것을 확인해야 하나?
## query keyword :  ['무보증 후순위사채', '기업실사', '기관', '계열회사', '확인사항']
## User query :  회사채의 무보증 후순위사채 기업실사를 규정하고 있는데, 기업실사에서 회사의 기관 및 계열회사에 관한 사항은 어떤 것을 확인해야 하나?
Keyword :  ['무보증 후순위사채', '기업실사', '기관', '계열회사', '확인사항']
## Total time taken: 0.042 seconds

## keyword triplets | total : 10 ##
(기관 -[SAME_AS]-> 중앙기록관리기관) IN 법_제117조의13제1항_중앙기록관리기관
(계열회사 -[INCLUDED_IN]-> 관계 투자매매업자) IN 시행령_제89조제2항_의결권행사의 제한 등
(계열회사 -[INCLUDED_IN]-> 임직원) IN 시행령_제84조_이해관계인의 범위
(계열회사 -[INCLUDED_IN]-> 대량취득ㆍ처분을 하려는 자) IN 법_제174조제3항_미공개중요정보 이용행위 금지
(계열회사 -[INCLUDED_IN]-> 공개매수예정자) IN 법_제174조제2항_미공개중요정보 이용행위 금지
(계열회사 -[INCLUDED_IN]-> 집합투자업자) IN 시행령_제209조_집합투자기구의 등록요건
(기관 -[INCLUDED_IN]-> 중앙기록관리기관대통령령으로 정하는 바에 따라 온라인소액투자중개업자로부터 온라인소액증권발행인과 투자자에 대한 정보를 제공받아 관리하는) IN 법_제117조의13제1항_중앙기록관리기관
(계열회사 -[INCLU

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


## Query : 회사채의 무보증 후순위사채 기업실사를 규정하고 있는데, 기업실사에서 회사의 기관 및 계열회사에 관한 사항은 어떤 것을 확인해야 하나?/n/n## Answer : 회사채의 무보증 후순위사채 발행과 관련하여 기업실사를 수행할 때, 회사의 기관 및 계열회사에 관한 사항을 꼼꼼히 확인해야 합니다. 일반적으로 확인해야 할 사항은 다음과 같습니다:

1. **소유 및 지배구조**: 발행 회사의 주주 구성, 주식 보유 비율, 그리고 지배구조가 명확히 이해되는지 확인합니다. 이는 대주주와 주요 이해관계자들이 누구인지 파악하는 데 중요합니다.

2. **계열회사 관계**: 주요 계열회사들의 목록과 이들 간의 관계를 파악합니다. 이는 특히 모회사, 자회사, 그리고 관계사의 경영성과가 발행 회사의 재무상황에 미치는 영향을 평가하는 데 필요합니다.

3. **재무상태**: 계열회사들의 재무상태와 성과를 검토해야 합니다. 이는 그룹 전체의 재정 건전성을 판단하는 데 도움이 됩니다.

4. **경영진 및 이사회의 구성**: 경영진 및 이사회 구성원의 배경과 역량, 그리고 그들의 의사결정 권한을 확인합니다. 이는 경영진의 안정성과 전문성을 평가하는 데 중요합니다.

5. **관련 거래**: 계열사 간의 주요 거래 내역(예: 자금 대여, 매출 및 매입 거래 등)을 파악하여, 거래가 공정하게 이루어졌는지, 또는 이러한 거래가 회사의 재무상태에 어떠한 영향을 미치는지 평가합니다.

6. **위험요인**: 계열회사 간의 리스크, 특히 재무적인 의존성 등이 중요한 요소로 인정될 수 있습니다. 이러한 위험요인이 계열사 또는 발행 회사에 어떤 영향을 미치는지를 분석합니다.

7. **기타 법적 의무 및 규제사항**: 계열회사와 관련된 법적 의무, 규제 리스크, 그리고 소송 등이 존재하는지 검토해야 합니다.

기업실사는 투자자에게 회사의 신용도를 정확히 평가할 수 있는 정보를 제공하는 중요한 과정입니다. 따라서, 이러한 사항들을 면밀히 조사하고 확인하는 것은 발행회사와 관련된 모든

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## Total time taken: 0.9228 seconds

## top-k : 5 | relevent nodes : 8
## relevant documents | total : 5##

####### idx: 0 #######
## VECTOR ## : Labels: Clause_01_law_main, Index: 제119조의2제2항, Similarity: 0.5555
## HOP ## : hops : 0, Labels: Clause_01_law_main, Index: 제119조의2제2항, Similarity: 0.5555

####### idx: 1 #######
## VECTOR ## : Labels: Clause_01_enforcement_main, Index: 제125조제1항, Similarity: 0.5361
## HOP ## : hops : 0, Labels: Clause_01_enforcement_main, Index: 제125조제1항, Similarity: 0.5361

####### idx: 2 #######
## VECTOR ## : Labels: Clause_01_enforcement_main, Index: 제328조제2항, Similarity: 0.5295
## HOP ## : hops : 0, Labels: Clause_01_enforcement_main, Index: 제328조제2항, Similarity: 0.5295

####### idx: 3 #######
## VECTOR ## : Labels: Clause_01_law_main, Index: 제117조의11제1항, Similarity: 0.5165
## HOP ## : hops : 0, Labels: Clause_01_law_main, Index: 제117조의11제1항, Similarity: 0.5165

####### idx: 4 #######
## VECTOR ## : Labels: Clause_01_enforcement_main, Index: 제324조의3제6항, S

## before

In [ ]:
# 단순 벡터 유사도 검색 결과와 hop 검색 결과를 얻음
top_k_results, related_docs = find_related_nodes_with_keywords(QUERY_EMB, keywords, config, hop=5)
answer_top_k_results, answer_related_docs = find_related_nodes_with_keywords(ANSWER_EMB, keywords, config, hop=5)
# hop 검색 결과의 길이를 config에 설정
config['top_k'] = len(related_docs)

# 수정된 top_k로 다시 단순 벡터 유사도 검색
top_k_results = find_similar_nodes(QUERY_EMB, dbms.driver, config)
# 수정된 top_k로 다시 단순 벡터 유사도 검색
answer_top_k_results = find_similar_nodes(ANSWER_EMB, dbms.driver, config)

# End timing
end_time = time.time()

# Calculate total time taken
total_time = end_time - start_time


print(f"Total time taken: {total_time:.2f} seconds\n")

# 인덱스별로 Top-K 검색 결과와 Hop 검색 결과를 한 번에 출력
print("\n========= Comparison of Hop-based and Vector-based Similarity Results =========")
max_length = min(len(top_k_results), len(related_docs))
print("length of top_k_results: ", len(top_k_results))

for idx in range(max_length):
    top_k_node = top_k_results[idx]
    related_doc = related_docs[idx]
    
    print(f"\n####### idx: {idx} #######")
    print(f"## VECTOR ## : Labels: {top_k_node['labels']}, Index: {top_k_node['index']}, Similarity: {top_k_node['similarity']:.4f}")
    print(f"## HOP ## : hops : {related_doc['hop']}, Labels: {related_doc['labels']}, Index: {related_doc['index']}, Similarity: {related_doc['similarity']:.4f}")

with open("../data/graph/retrieval/hop-retrieve_results.json", "w") as f:
    json.dump(related_docs, f, indent=4, ensure_ascii=False)
    
# top_k_results, answer_top_k_results 를 json 형태 저장 
with open("../data/graph/retrieval/top_k_results.json", "w") as f:
    json.dump(top_k_results, f, indent=4, ensure_ascii=False)

with open("../data/graph/retrieval/answer_top_k_results.json", "w") as f:
    json.dump(answer_top_k_results, f, indent=4, ensure_ascii=False)

with open("../data/graph/retrieval/answer_related_docs.json", "w") as f:
    json.dump(answer_related_docs, f, indent=4, ensure_ascii=False)

In [ ]:

vector_node_ids = set([node['node_id'] for node in top_k_results])
hop_node_ids = set([doc['node_id'] for doc in related_docs])

# 벡터 기반 검색 결과에는 있지만 hop 기반 검색 결과에는 없는 노드
only_in_vector = vector_node_ids - hop_node_ids
print(f"\n####### Node in Vector-based Only #######")
for idx, node in enumerate(top_k_results):
    if node['node_id'] in only_in_vector:
        
        print(f"idx : {idx}, Labels: {node['labels']}, Index: {node['index']}, Similarity: {node['similarity']:.4f}")


print(f"\n####### Node in Hop-based Only #######")
# hop 기반 검색 결과에는 있지만 벡터 기반 검색 결과에는 없는 노드
only_in_hop = hop_node_ids - vector_node_ids
for idx, doc in enumerate(related_docs):
    if doc['node_id'] in only_in_hop:
        
        print(f"idx : {idx}, Labels: {doc['labels']}, Index: {doc['index']}, Similarity: {doc['similarity']:.4f}")


In [ ]:

# query -> vector embedding , keyword 추출 
query = "증권신고서 제출기한 연장에 대한 법적 책임은 무엇인가요?"
query_embedding_vector = utils.text_embed(text = query, embed_model = config.get('embedding_model') )
query_keywords = utils.extract_keyword(text=query, prompt_path = config["query_keyword_prompt_path"],keyword_model = config["model"])
print(f"## query : {query}, query_keywords : {query_keywords}, len(query_embedding_vector) : {len(query_embedding_vector)}")



# .py 위한 재료 


In [ ]:


def main(config: Dict):
    
    parser = argparse.ArgumentParser()
    g = parser.add_argument_group("Settings")
    g.add_argument("--topK", type=int, default=10, help="Number of top related documents to retrieve")
    g.add_argument("--threshold", type=float, default=0.5, help="Threshold for relatedness score")
    g.add_argument("--output", type=str, default="C:/Users/Shic/development/GIB/legal_graph-main/data/graph/clause/retrieve/related_documents.json", help="Output file path")
    g.add_argument("--query" , type=str, default="C:/Users/Shic/development/GIB/legal_graph-main/data/graph/clause/retrieve/query.txt", help="Query file path")
    

    args = parser.parse_args()
    database = config.get("database")

    
    dbms = LegalGraphDB(auradb=False, config=config)

    # query -> vector embedding , keyword 추출 
    query_embedding_vector = utils.text_embed(config, args.query)
    query_keywords = extract_keyword(config, args.query)
    
    # 전체 node 중 query와 가장 유사한 topK개의 node_id를 반환
    retrieved_nodes = retrieve_related_documents(dbms, config , embed_vec = query_embedding_vector, keyword = query_keywords) 

    
    
    # topk 노드와 간선으로 연결된 node 중 (유사도가 threshold 이상) & (query에서 추출한 keyword를 포함하고 있으면서 ) hop1의 node_id를 반환
    # hopH까지 반복 
    # 반환된 document 에서 reranking -> 키워드 포함 / 유사도 / 홉 수 등 고려하여 가중치 부여 
    # reranking된 노드를 json으로 저장 
    # ex. {query : str , retrievedDocuments: {node_id : {keyword : [keyword1, keyword2, ...], similarity : 0.8, hop : 2}}} 


if __name__ == "__main__":
    # config.json 파일 경로를 절대 경로로 설정
    config_path = os.path.join(root_path, 'codes', 'configs', 'config.json')
    with open(config_path, 'r') as f:
        config = json.load(f)
    
    main(config=config)
